In [3]:
from skmultilearn.dataset import load_dataset
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import re
import random
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import os
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.dataset import load_from_arff
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import sklearn.metrics as metrics
from torch.utils.data import DataLoader, Subset
import itertools
import pickle
from sklearn.preprocessing import MinMaxScaler
from skmultilearn.dataset import load_dataset
from skmultilearn.model_selection import IterativeStratification
from sklearn.neighbors import NearestNeighbors
import pdb
import scipy
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from enum import Enum
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from torch.optim.lr_scheduler import LambdaLR
from util import *
from layers import  *
from model import *
from function import *
from scipy.stats import pearsonr
from scipy.stats import pearsonr
from collections import deque
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity
from scipy.integrate import dblquad
from joblib import Parallel, delayed
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy

In [4]:
#####seed####
def seed_all(seed): 
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed=42
seed_all(seed)
device = torch.device("cuda:0" ) if torch.cuda.is_available() else torch.device("cpu")

In [5]:
class CFG:
    def __init__(self,name,X,y):
        self.name=name      
        self.X_train=X
        self.y_train=y
        self.configs={} 
    def getconfig(self):
        self.configs['label_matrix']=np.array(self.y_train)
        self.configs['num_classes']=self.y_train.shape[1] 
        self.configs['num_ins']=self.X_train.shape[0] 
        self.configs['seed']=42 
        self.configs['batch_size']=128
        self.configs['epoch']=100 
        self.configs['lr']=1e-4
        self.configs['device']=torch.device("cuda:0" ) if torch.cuda.is_available() else torch.device("cpu")
        self.configs['weight']=limb(np.array(self.X_train),np.array(self.y_train))
        self.configs['extra_sample']=int(self.X_train.shape[0]*0.1)
        self.configs['min_ins_idx']=minority_instance(np.array(self.y_train))
        self.configs['minority_label_indices'],_=Labeltype(np.array(self.y_train))
        self.configs['weight_list']=calweight(np.array(self.X_train),np.array(self.y_train))
        self.configs['card'],_=CardAndDens(np.array(self.X_train),np.array(self.y_train))
        #DELA
        if self.name=='DELA':
            self.configs['in_features']=self.X_train.shape[1] 
            self.configs['latent_dim']=math.ceil(self.X_train.shape[1]/2)    
            self.configs['lr_ratio']=0.8
            self.configs['drop_ratio']=0.2
            self.configs['tau']=2/3
            self.configs['beta']=1e-4
            self.configs['out_index']=-1
        if self.name=='CLIF':
            self.configs['class_emb_size']=self.y_train.shape[1]  
            self.configs['input_x_size']=self.X_train.shape[1] 
            self.configs['num_layers']=2 
            self.configs['in_layers']=3 
            self.configs['hidden_list']=[math.ceil(self.y_train.shape[1]/2)]  
            self.configs['out_index']=0        
        if self.name=='PACA':
            self.configs['drop_ratio']=0.1
            self.configs['latent_dim']=math.ceil(self.X_train.shape[1]/2)
            self.configs['in_features']=self.X_train.shape[1] #输入x的维度
            self.configs['rand_seed']=self.configs['seed']
            self.configs['eps']=1e-8    
            self.configs['lr_scheduler']='fix'
            self.configs['binary_data']=False
            self.configs['weight_decay']=1e-5
            self.configs['alpha']=2
            self.configs['gamma']=10
            self.configs['scheduler_warmup_epoch']=5
            self.configs['scheduler_decay_epoch']=10
            self.configs['scheduler_decay_rate']=1e-5 
            self.configs['out_index']=-2
        return self.configs

In [6]:
seed_all(seed)
device = torch.device('cuda')
def FeatureSelect(X,p):
    if p==1:
        return X.toarray(),feature_names
    else:
        featurecount=int(X.shape[1]*p)
        Selectfeatureindex=[x[0] for x in (sorted(enumerate(X.sum(axis=0).tolist()[0]),key=lambda x: x[1],reverse=True))][:featurecount]
        Allfeatureindex=[i for i in range(X.shape[1])]
        featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
        new_x=np.delete(X.toarray(),featureindex,axis=1)
        new_featurename=[feature_names[i] for i in Selectfeatureindex] 
        return new_x,new_featurename
def LabelSelect(y):
    b=[]
    new_labelname=[i for i in label_names]
    for i in range(y.shape[1]):
        if y[:,i].sum()<=20:
            b.append(i)
            new_labelname.remove(label_names[i])
    new_y=np.delete(y.toarray(),b,axis=1)
    return new_y,new_labelname
def macro_averaging_auc(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(l)
    q = np.sum(Y, 0)

    zero_column_count = np.sum(q == 0)
#     print(f"all zero for label: {zero_column_count}")
    r, c = np.nonzero(Y)
    for i, j in zip(r, c):
        p[j] += np.sum((Y[ : , j] < 0.5) * (O[ : , j] <= O[i, j]))

    i = (q > 0) * (q < n)

    return np.sum(p[i] / (q[i] * (n - q[i]))) / l
def hamming_loss(Y, P, O):
    n = (Y.shape[0] + P.shape[0]) // 2
    l = (Y.shape[1] + P.shape[1]) // 2

    s1 = np.sum(Y, 1)
    s2 = np.sum(P, 1)
    ss = np.sum(Y * P, 1)

    return np.sum(s1 + s2 - 2 * ss) / (n * l)
def one_error(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2

    i = np.argmax(O, 1)

    return np.sum(1 - Y[range(n), i]) / n
def ranking_loss(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(n)
    q = np.sum(Y, 1)

    r, c = np.nonzero(Y)
    for i, j in zip(r, c): 
        p[i] += np.sum((Y[i, : ] < 0.5) * (O[i, : ] >= O[i, j]))

    i = (q > 0) * (q < l)

    return np.sum(p[i] / (q[i] * (l - q[i]))) / n
def micro_f1(Y, P, O):
    return f1_score(Y, P, average='micro')
def macro_f1(Y, P, O):
    return f1_score(Y, P, average='macro')
def eval_metrics(mod, metrics, datasets, idx,batch_size,device):
    res_dict = {}
    mod.eval()
    y_true_list = []
    y_scores_list = []
    test_dataloader = DataLoader(datasets, batch_size=batch_size, shuffle=True, num_workers=0)
    for x, y in test_dataloader:
        _, y_pred=mod.predict(x)
#         pdb.set_trace()
        y_true_list.append(y.cpu().numpy())
        y_scores_list.append(y_pred.cpu().numpy())
    y_true = np.vstack(y_true_list)
    y_prob = np.vstack(y_scores_list)
    y_pred = np.round(y_prob).astype(int)
    res_dict1 = {metric.__name__: metric(y_true, y_pred,y_prob) for metric in metrics}
#         # Calculate metric.
#         res_dict1 = {metric.__name__: metric(y_true, y_pred) for metric in metrics[:2]}
#         res_dict2 = {metric.__name__: metric(y_true, y_prob) for metric in metrics[2:5]}
#         res_dict1.update(res_dict2)
#     res_dict[f'dataset_{ix}']=res_dict1
#         res_dict[f'dataset_{ix}'] = {metric.__name__: metric(y_true, y_pred) for metric in metrics[:8]}
#         res_dict[f'dataset_{ix}'] = {metric.__name__: metric(y_true, y_prob) for metric in metrics[8:9]}
    return res_dict1

# Random

In [5]:
 def training(configs,test_epoch):
    net=CLIFModel(configs).to(device)
    num_epochs =configs['epoch']
    batch_size = configs['batch_size']
    lr = configs['lr']
    label_dim=configs['num_classes']
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    writer = SummaryWriter(comment=f'{dataname}')

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    validation_dataset, test_dataset_new = train_test_split(test_dataset, test_size=0.5, random_state=42)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    best_auc = 0
    best_epoch=0
    best_model_state = None
    epoch_losses_train = []
    batch_losses = []
    epoch_losses_val= []
    all_loss_list=[]
    total_minority_samples = len(configs['min_ins_idx'])
    
    warmup_epochs =5
    total_steps = num_epochs * int(configs['num_ins']/batch_size)
    warmup_steps = warmup_epochs * int(configs['num_ins']/batch_size)
    global_step = 0
    def update_learning_rate(optimizer, global_step, warmup_steps=warmup_steps, base_lr=lr):
        if global_step < warmup_steps:
            lr = base_lr * (global_step / warmup_steps)
        else:
            lr = base_lr
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr 
     
    for epoch in range(num_epochs+1): 
        net.train()
        loss_tracker = 0.0
        all_indices = [] 
        batch_counter = 0       
        for x, y in train_dataloader:
            optimizer.zero_grad()
            outputs = net(x)
            loss_dict = net.loss_function_train(outputs, y)
            loss = loss_dict['Loss']
            individual_losses =net._loss_per_label(outputs[configs['out_index']], y)
            if epoch==test_epoch:
                for i in range(individual_losses.shape[0]):
                    all_loss_list.append(round(individual_losses[i].item(), 3))
            loss.backward()
            optimizer.step()
            global_step += 1
            update_learning_rate(optimizer, global_step) 
            loss_tracker += loss.item()
            batch_counter += 1

        writer.add_scalar('train/loss', loss_tracker, epoch)
        epoch_losses_train.append(loss_tracker /batch_counter)  
        
        
        
        net.eval()
        y_true_list = []
        y_scores_list = []
        loss_tracker = 0.0
        batch_counter = 0
        with torch.no_grad():
            for x, y in validation_dataloader:
                _, y_pred=net.predict(x)
                y_true_list.append(y.cpu().numpy())
                y_scores_list.append(y_pred.cpu().numpy())
                
                outputs = net(x)
                loss_dict = net.loss_function_train(outputs, y)
                loss = loss_dict['Loss']
                loss_tracker += loss.item()
                batch_counter += 1
        epoch_losses_val.append(loss_tracker /batch_counter)
        y_true = np.vstack(y_true_list)
        y_scores = np.vstack(y_scores_list)
        auc = macro_averaging_auc(y_true,y_scores, y_scores)
        writer.add_scalar('val/auc', auc, epoch)
        if auc > best_auc:
            best_auc = auc
            best_epoch=epoch
            best_model_state = net.state_dict().copy()
    net.load_state_dict(best_model_state)
    mets = eval_metrics(net, [macro_f1, micro_f1, macro_averaging_auc, ranking_loss, hamming_loss, one_error], test_dataset_new, configs['out_index'],configs['batch_size'],torch.device('cuda'))
    return all_loss_list

# path_to_arff_files = ["Corel5k","bibtex"]
# label_counts = [374,159]
# select_feature=[1,1]
# sample_loss_dict={}

path_to_arff_files = ["yahoo-Business1"]
label_counts = [28]
select_feature=[0.02]
sample_loss_dict={}

for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    X, y, feature_names, label_names = load_from_arff(
    path_to_arff_file,
    label_count=label_counts[idx],
    label_location="end",
    load_sparse=False,
    return_attribute_definitions=True
    )
    X,feature_names=FeatureSelect(X,select_feature[idx])  
    y,label_names=LabelSelect(y)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    k_fold = IterativeStratification(n_splits=5,order=1,random_state=42)
    dicts=[]
    for idx,(train,test) in enumerate(k_fold.split(X,y)):
        if idx==4:
            train_dataset = TensorDataset(torch.tensor(X[train], device=device, dtype=torch.float),torch.tensor(y[train], device=device,dtype=torch.float))
            test_dataset = TensorDataset(torch.tensor(X[test], device=device, dtype=torch.float), torch.tensor(y[test], device=device, dtype=torch.float))    
            configs=CFG('CLIF',X[train],y[train]).getconfig()
            result=training(configs,70)
            sample_loss_dict[dataname]=result
#             epoch_loss_dict_train[dataname]=epoch_losses_train
#             epoch_loss_dict_val[dataname]=epoch_losses_val
#             epoch_auc_dict_val[dataname]=epoch_auc_val
#             print(best_epoch)

import pickle
# with open('epoch_loss_dict_train0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_train, f)
# with open('epoch_loss_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_val, f)
# with open('epoch_auc_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_auc_dict_val, f)
# with open('batch_loss_dict_train1.pickle', 'wb') as f:
#     pickle.dump(batch_loss_dict_train, f)
with open('Figure/sample_loss_dict_Base(70).pickle', 'wb') as f:
    pickle.dump(sample_loss_dict, f)

# Balance

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

class StratifiedBatchSampler:
    def __init__(self, labels, batch_size):
        self.labels = labels
        self.batch_size = batch_size
        self.indices = np.arange(len(labels))
        self.unique_labels, self.label_counts = np.unique(labels, axis=0, return_counts=True)

    def __iter__(self):
        batch_indices = []
        label_indices = {tuple(label): np.where((self.labels == label).all(axis=1))[0].tolist() for label in self.unique_labels}
        
        while len(self.indices) >= self.batch_size:
            for label, count in zip(self.unique_labels, self.label_counts):
                if len(batch_indices) >= self.batch_size:
                    break
                label_tuple = tuple(label)
                if len(label_indices[label_tuple]) > 0:
                    chosen_indices = np.random.choice(label_indices[label_tuple], size=min(count, self.batch_size - len(batch_indices)), replace=False)
                    batch_indices.extend(chosen_indices)
                    label_indices[label_tuple] = [i for i in label_indices[label_tuple] if i not in chosen_indices]

            if len(batch_indices) == self.batch_size:
                yield batch_indices
                batch_indices = []

        if len(batch_indices) > 0:
            yield batch_indices

    def __len__(self):
        return len(self.indices) // self.batch_size

class CustomDataloader:
    def __init__(self, dataset, batch_size, shuffle=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = torch.arange(len(dataset), dtype=torch.long)  # 确保索引为长整型

    def set_epoch(self, epoch):
        if self.shuffle:
            self.indices = torch.randperm(len(self.dataset), dtype=torch.long)  # 确保索引为长整型

    def __iter__(self):
        sampler = torch.utils.data.BatchSampler(
            torch.utils.data.SubsetRandomSampler(self.indices),
            batch_size=self.batch_size,
            drop_last=False
        )

        for batch_indices in sampler:
            yield self.get_data_from_indices(batch_indices)

    def get_data_from_indices(self, indices):
        x, y = zip(*[self.dataset[i] for i in indices])
        return indices, torch.stack(x), torch.stack(y)

# 示例训练函数
def training(configs, test_epoch):
    net = CLIFModel(configs).to(device)
    num_epochs = configs['epoch']
    batch_size = configs['batch_size']
    lr = 1e-4
    label_dim = configs['num_classes']
    weight_decay = 1e-4
    betas = (0.9, 0.999)

    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
    writer = SummaryWriter(comment=f'{dataname}')

    sample_probabilities = {}
    para_loss = {key: 0 for key in range(configs['num_ins'])}
    custom_dataloader = CustomDataloader(train_dataset, batch_size=batch_size)
    validation_dataset, test_dataset_new = train_test_split(test_dataset, test_size=0.5, random_state=42)
    test_dataloader = DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False, num_workers=0)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    best_auc = 0
    best_model_state = None
    epoch_losses_train = []
    all_loss_list = []

    warmup_epochs = 5
    total_steps = num_epochs * int(configs['num_ins'] / batch_size)
    warmup_steps = warmup_epochs * int(configs['num_ins'] / batch_size)
    global_step = 0

    def update_learning_rate(optimizer, global_step, warmup_steps=warmup_steps, base_lr=lr):
        if global_step < warmup_steps:
            lr = base_lr * (global_step / warmup_steps)
        else:
            lr = base_lr
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    for epoch in range(num_epochs + 1):
        net.train()
        batch_counter = 0
        loss_tracker = 0.0
        loss_tracker2 = 0.0
        all_individual_losses = {}
        custom_dataloader.set_epoch(epoch)
        for idx, x, y in custom_dataloader:
            optimizer.zero_grad()
            outputs = net(x)
            loss_dict = net.loss_function_train(outputs, y)
            loss = loss_dict['Loss']
            individual_losses1 = net._loss_per_label(outputs[configs['out_index']], y)
            if epoch == test_epoch:
                for i in range(individual_losses1.shape[0]):
                    all_loss_list.append(round(individual_losses1[i].item(), 3))
            loss.backward()
            optimizer.step()
            global_step += 1
            update_learning_rate(optimizer, global_step)
            batch_counter += 1
        writer.add_scalar('train/loss', loss_tracker, epoch)
        epoch_losses_train.append(loss_tracker / batch_counter)

        net.eval()
        y_true_list = []
        y_scores_list = []
        loss_tracker = 0.0
        batch_counter = 0
        with torch.no_grad():
            for x, y in validation_dataloader:
                _, y_pred = net.predict(x)
                y_true_list.append(y.cpu().numpy())
                y_scores_list.append(y_pred.cpu().numpy())

                outputs = net(x)
                loss_dict = net.loss_function_train(outputs, y)
                loss = loss_dict['Loss']
                loss_tracker += loss.item()
                batch_counter += 1
        y_true = np.vstack(y_true_list)
        y_scores = np.vstack(y_scores_list)

        auc = macro_averaging_auc(y_true, y_scores, y_scores)

        if auc > best_auc:
            best_auc = auc
            best_epoch = epoch
            best_model_state = net.state_dict().copy()
    net.load_state_dict(best_model_state)
    mets = eval_metrics(net, [macro_f1, micro_f1, macro_averaging_auc, ranking_loss, hamming_loss, one_error], test_dataset_new, configs['out_index'], configs['batch_size'], torch.device('cuda'))

    return all_loss_list

path_to_arff_files = ["emotions","scene","yeast", "Corel5k","rcv1subset1","rcv1subset2","rcv1subset3","yahoo-Business1","yahoo-Arts1","bibtex",'tmc2007','enron','cal500','LLOG-F']
label_counts = [6, 6,14,374,101,101,101,28,25,159,22,53,174,75]
select_feature=[1,1,1,1,0.02,0.02,0.02,0.05,0.05,1,0.01,1,1,1]

path_to_arff_files = ["Corel5k","bibtex"]
label_counts = [374,159]
select_feature=[1,1]
sample_loss_dict={}

path_to_arff_files = ["yahoo-Business1"]
label_counts = [28]
select_feature=[0.02]
sample_loss_dict={}

for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    X, y, feature_names, label_names = load_from_arff(
    path_to_arff_file,
    label_count=label_counts[idx],
    label_location="end",
    load_sparse=False,
    return_attribute_definitions=True
    )
    X,feature_names=FeatureSelect(X,select_feature[idx])  
    y,label_names=LabelSelect(y)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    warm_epoch=3
    print(dataname)
    k_fold = IterativeStratification(n_splits=5,order=1,random_state=42)
    dicts=[]
    for idx,(train,test) in enumerate(k_fold.split(X,y)):
        if idx==4:
            train_dataset = TensorDataset(torch.tensor(X[train], device=device, dtype=torch.float),torch.tensor(y[train], device=device,dtype=torch.float))
            test_dataset = TensorDataset(torch.tensor(X[test], device=device, dtype=torch.float), torch.tensor(y[test], device=device, dtype=torch.float))    
            configs=CFG('CLIF',X[train],y[train]).getconfig()
            result=training(configs,70)
            sample_loss_dict[dataname]=result
#             epoch_loss_dict_train[dataname]=epoch_losses_train
#             epoch_loss_dict_val[dataname]=epoch_losses_val
#             epoch_auc_dict_val[dataname]=epoch_auc_val
#             print(best_epoch)
# with open('epoch_loss_dict_train0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_train, f)
# with open('epoch_loss_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_val, f)
# with open('epoch_auc_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_auc_dict_val, f)
# with open('batch_loss_dict_train1.pickle', 'wb') as f:
#     pickle.dump(batch_loss_dict_train, f)
with open('Figure/sample_loss_dict_Balance(70).pickle', 'wb') as f:
    pickle.dump(sample_loss_dict, f)

yahoo-Business1


# Hard

In [6]:
def calculate_probabilities(losses_dict,se):
    losses_np = np.array(list(losses_dict.values()))
    Delta=max(losses_np)/len(losses_np)
    se = se 
    N = len(losses_np)
    quantized_indices = np.ceil(losses_np / Delta).astype(int)
    probabilities = (np.exp(np.log(se) / N)) ** quantized_indices
    probabilities_dict = {key: prob for key, prob in zip(losses_dict.keys(), probabilities)}
    
    return probabilities_dict
class CustomDataLoader:
    def __init__(self, dataset, net, configs,sample_probabilities,warm_epoch):
        self.shuffle = False
        self.dataset = dataset
        self.net = net
        self.batch_size = configs['batch_size']
        self.sample_probabilities =sample_probabilities
        self.dataset_indices=list(range(len(self.dataset)))
        self.warm_epoch=warm_epoch       
    def __iter__(self):
        if self.current_epoch < self.warm_epoch:
            batch_counter = 0
            for start_idx in range(0, len(self.dataset_indices), self.batch_size):
                end_idx = min(start_idx + self.batch_size, len(self.dataset_indices))
                batch_indices = self.dataset_indices[start_idx:end_idx]
                batch_counter += 1
                yield self.get_data_from_indices(batch_indices)
        else:
            all_indices = list(self.sample_probabilities.keys())        
            all_probabilities = np.array([self.sample_probabilities[idx] for idx in all_indices])
            non_zero_mask = all_probabilities != 0
            non_zero_sum = all_probabilities[non_zero_mask].sum()
            all_probabilities[non_zero_mask] /= non_zero_sum
            total_samples = len(self.dataset)
            num_batches = np.ceil(total_samples / self.batch_size).astype(int)

            for _ in range(num_batches):
                num_required = self.batch_size 
                if num_required > 0:
                    chosen_indices = np.random.choice(all_indices, size=num_required, replace=False, p=all_probabilities)
                else:
                    chosen_indices = []
                batch_indices = list(chosen_indices)
                yield self.get_data_from_indices(batch_indices)
    def get_data_from_indices(self, indices):
        x, y = zip(*[self.dataset[i] for i in indices])
        return indices, torch.stack(x), torch.stack(y)
    def set_epoch(self, epoch):
        self.current_epoch = epoch  
def training(configs,test_epoch):
    net=CLIFModel(configs).to(device)
    num_epochs =configs['epoch']
    batch_size = configs['batch_size']
    lr = 1e-4
    label_dim=configs['num_classes']
    weight_decay = 1e-4  
    betas = (0.9, 0.999) 

    optimizer = torch.optim.Adam(net.parameters(), lr=lr,  
                           betas=betas, weight_decay=weight_decay)
    writer = SummaryWriter(comment=f'{dataname}')

    sample_probabilities = {}
    para_loss={key: 0 for key in range(configs['num_ins'])}
    custom_dataloader = CustomDataLoader(train_dataset, net=net, configs=configs,sample_probabilities=sample_probabilities,warm_epoch=warm_epoch)
    validation_dataset, test_dataset_new = train_test_split(test_dataset, test_size=0.5, random_state=42)   
    test_dataloader = DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False, num_workers=0)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    best_auc = 0
    best_model_state = None
    epoch_losses_train = []
    all_loss_list=[]

    warmup_epochs =5
    total_steps = num_epochs * int(configs['num_ins']/batch_size)
    warmup_steps = warmup_epochs * int(configs['num_ins']/batch_size)
    global_step = 0
    def update_learning_rate(optimizer, global_step, warmup_steps=warmup_steps, base_lr=lr):
        if global_step < warmup_steps:
            lr = base_lr * (global_step / warmup_steps)
        else:
            lr = base_lr
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr         
    for epoch in range(num_epochs+1): 
        net.train()
        batch_counter = 0   
        loss_tracker = 0.0
        loss_tracker2=0.0
        all_individual_losses = {}
        custom_dataloader.set_epoch(epoch) 
        for idx, x, y in custom_dataloader:
            optimizer.zero_grad()
            outputs = net(x)
            loss_dict = net.loss_function_train(outputs, y)
            loss = loss_dict['Loss']
            individual_losses =net.custom_multilabel_soft_margin_loss(outputs[configs['out_index']], y,  [configs['weight_list'][i] for i in idx])
            
            individual_losses1 =net._loss_per_label(outputs[configs['out_index']], y)
            if epoch==test_epoch:
                for i in range(individual_losses1.shape[0]):
                    all_loss_list.append(round(individual_losses1[i].item(), 3))
            loss.backward()
            optimizer.step()
            global_step += 1
            update_learning_rate(optimizer, global_step)

            for i, prob in zip(idx, individual_losses):
                para_loss[i]=prob.item()
            current_probabilities = calculate_probabilities(para_loss,8)
            for i in idx:
                if i in current_probabilities:
                    sample_probabilities[i] = current_probabilities[i]

            custom_dataloader.sample_probabilities = sample_probabilities
            batch_counter+=1
        writer.add_scalar('train/loss', loss_tracker, epoch)
        epoch_losses_train.append(loss_tracker /batch_counter)

        net.eval()
        y_true_list = []
        y_scores_list = []
        loss_tracker = 0.0
        batch_counter = 0
        with torch.no_grad():
            for x, y in validation_dataloader:
                _, y_pred=net.predict(x)
                y_true_list.append(y.cpu().numpy())
                y_scores_list.append(y_pred.cpu().numpy())
                
                outputs = net(x)
                loss_dict = net.loss_function_train(outputs, y)
                loss = loss_dict['Loss']
                loss_tracker += loss.item()
                batch_counter += 1
        y_true = np.vstack(y_true_list)
        y_scores = np.vstack(y_scores_list)

        auc = macro_averaging_auc(y_true,y_scores, y_scores)

        if auc > best_auc:
            best_auc = auc
            best_epoch=epoch
            best_model_state = net.state_dict().copy()
    net.load_state_dict(best_model_state)
    mets = eval_metrics(net, [macro_f1, micro_f1, macro_averaging_auc, ranking_loss, hamming_loss, one_error], test_dataset_new, configs['out_index'],configs['batch_size'],torch.device('cuda'))

    return all_loss_list


path_to_arff_files = ["emotions","scene","yeast", "Corel5k","rcv1subset1","rcv1subset2","rcv1subset3","yahoo-Business1","yahoo-Arts1","bibtex",'tmc2007','enron','cal500','LLOG-F']
label_counts = [6, 6,14,374,101,101,101,28,25,159,22,53,174,75]
select_feature=[1,1,1,1,0.02,0.02,0.02,0.05,0.05,1,0.01,1,1,1]

path_to_arff_files = ["Corel5k","bibtex"]
label_counts = [374,159]
select_feature=[1,1]
sample_loss_dict={}

path_to_arff_files = ["yahoo-Business1"]
label_counts = [28]
select_feature=[0.02]
sample_loss_dict={}

for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    X, y, feature_names, label_names = load_from_arff(
    path_to_arff_file,
    label_count=label_counts[idx],
    label_location="end",
    load_sparse=False,
    return_attribute_definitions=True
    )
    X,feature_names=FeatureSelect(X,select_feature[idx])  
    y,label_names=LabelSelect(y)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    warm_epoch=3
    print(dataname)
    k_fold = IterativeStratification(n_splits=5,order=1,random_state=42)
    dicts=[]
    for idx,(train,test) in enumerate(k_fold.split(X,y)):
        if idx==4:
            train_dataset = TensorDataset(torch.tensor(X[train], device=device, dtype=torch.float),torch.tensor(y[train], device=device,dtype=torch.float))
            test_dataset = TensorDataset(torch.tensor(X[test], device=device, dtype=torch.float), torch.tensor(y[test], device=device, dtype=torch.float))    
            configs=CFG('CLIF',X[train],y[train]).getconfig()
            result=training(configs,70)
            sample_loss_dict[dataname]=result
#             epoch_loss_dict_train[dataname]=epoch_losses_train
#             epoch_loss_dict_val[dataname]=epoch_losses_val
#             epoch_auc_dict_val[dataname]=epoch_auc_val
#             print(best_epoch)
# with open('epoch_loss_dict_train0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_train, f)
# with open('epoch_loss_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_val, f)
# with open('epoch_auc_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_auc_dict_val, f)
# with open('batch_loss_dict_train1.pickle', 'wb') as f:
#     pickle.dump(batch_loss_dict_train, f)
# with open('Figure/sample_loss_dict_Hard(70).pickle', 'wb') as f:
#     pickle.dump(sample_loss_dict, f)

yahoo-Business1


# Active

In [ ]:
def std_H(H):
    var_H = np.var(H, ddof=1)  # ddof=1 for sample variance
    std = np.sqrt(var_H + var_H**2 / (len(H) - 1))
    return std
def update_H(H, y_pred, ids, max_history_length=epoch):
    y_pred_numpy = y_pred.detach().cpu().numpy()
    for i, idx in enumerate(ids):
        if idx not in H:
            H[idx] = deque(maxlen=max_history_length) 
        H[idx].append(y_pred_numpy[i])  
    return H
def update_E(H,E,ids,label_dim):
    for idx in ids:
        current_predictions_history = np.array(H[idx])
        for j in range(label_dim):      
            E[idx,j]=std_H(current_predictions_history[:, j])
    return E

def update_U(E, U, epoch):
    U = np.sum(E, axis=1)
    return U

def calculate_probabilities(U, epoch, e_end, e0, se0):
    N=len(U)
    Delta = 1 / N 
    # Exponentially decay the selection pressure as training progresses
    se = se0 * (np.exp(np.log(1 / se0) / (e_end - e0)))**(epoch - e0)
    U_min = np.min(U)
    U_max = np.max(U)
    U_normalized = (U - U_min) / (U_max - U_min)
    quantized_indices = np.ceil((1 - U_normalized) / Delta).astype(int)
#     pdb.set_trace()
    # Calculate the exponent for the sampling probability
    exponent = -np.log(se) / N * quantized_indices 
    # Calculate the unnormalized probabilities
    unnormalized_probabilities = np.exp(exponent)
    return unnormalized_probabilities

class CustomDataLoader:
    def __init__(self, dataset, net, configs,H,E,U,sample_probabilities,warm_epoch):
        self.shuffle = False
        self.dataset = dataset
        self.net = net
        self.batch_size = configs['batch_size']
        self.sample_probabilities =sample_probabilities
        self.dataset_indices=list(range(len(self.dataset)))
        self.warm_epoch=warm_epoch   
        
        self.H=H
        self.E=E
        self.U=U    
                              
    def __iter__(self):
        if self.current_epoch < self.warm_epoch:
            batch_counter = 0
            for start_idx in range(0, len(self.dataset_indices), self.batch_size):
                end_idx = min(start_idx + self.batch_size, len(self.dataset_indices))
                batch_indices = self.dataset_indices[start_idx:end_idx]
                batch_counter += 1
                yield self.get_data_from_indices(batch_indices)
        else:
            all_indices = self.dataset_indices  
            all_probabilities = self.sample_probabilities   

            total_sum = all_probabilities.sum()
            all_probabilities /= total_sum

            total_samples = len(self.dataset)
            num_batches = np.ceil(total_samples / self.batch_size).astype(int)

            for _ in range(num_batches):
                num_required = self.batch_size 
                if num_required > 0:
                    chosen_indices = np.random.choice(all_indices, size=num_required, replace=False, p=all_probabilities)
                else:
                    chosen_indices = []
                batch_indices = list(chosen_indices)
                yield self.get_data_from_indices(batch_indices)
                
    def get_data_from_indices(self, indices):
        x, y = zip(*[self.dataset[i] for i in indices])
        return indices, torch.stack(x), torch.stack(y)
    def set_epoch(self, epoch):
        self.current_epoch = epoch

# Recent

In [ ]:
def entropy(H_ij_t):
    classes = [0, 1]
    T = len(H_ij_t)
    probabilities = []
#     class 1
    count_1 = sum([1 for y in H_ij_t if y > 0.5])
    P_1 = count_c / T
    probabilities.append(P_1)
#    class 0
    P_0= 1-P_1
    probabilities.append(P_0)
    entropy_value = -sum(P_c * np.log2(P_c) for P_c in probabilities if P_c > 0)  
    return entropy_value
# 队列函数
def update_H(H, y_pred, ids, max_history_length=5):
# y_pred_numpy[batch*q]   批样本ids在原训练集的id
    y_pred_numpy = y_pred.detach().cpu().numpy()
    for i, idx in enumerate(ids):
        if idx not in H:
            H[idx] = deque(maxlen=max_history_length) 
        H[idx].append(y_pred_numpy[i])  
    return H
# 每个标签不确定性度量的中间函数
def update_E(H,E,ids,label_dim):
    for idx in ids:
# current_predictions_history[max_history_length*q]
        current_predictions_history = np.array(H[idx])
        for j in range(label_dim):      
            E[idx,j]=entropy(current_predictions_history[:, j])
    return E

def update_U(E, U, epoch):
    U = np.sum(E, axis=1)
    return U

def calculate_probabilities(U, epoch, e_end, e0, se0):
    N=len(U)
    Delta = 1 / N 
    # Exponentially decay the selection pressure as training progresses
    se = se0 * (np.exp(np.log(1 / se0) / (e_end - e0)))**(epoch - e0)
    U_min = np.min(U)
    U_max = np.max(U)
    U_normalized = (U - U_min) / (U_max - U_min)
    quantized_indices = np.ceil((1 - U_normalized) / Delta).astype(int)
#     pdb.set_trace()
    # Calculate the exponent for the sampling probability
    exponent = -np.log(se) / N * quantized_indices 
    # Calculate the unnormalized probabilities
    unnormalized_probabilities = np.exp(exponent)
    return unnormalized_probabilities

class CustomDataLoader:
    def __init__(self, dataset, net, configs,H,E,U,sample_probabilities,warm_epoch):
        self.shuffle = False
        self.dataset = dataset
        self.net = net
        self.batch_size = configs['batch_size']
        self.sample_probabilities =sample_probabilities
        self.dataset_indices=list(range(len(self.dataset)))
        self.warm_epoch=warm_epoch   
        
        self.H=H
        self.E=E
        self.U=U    
                              
    def __iter__(self):
        if self.current_epoch < self.warm_epoch:
            batch_counter = 0
            for start_idx in range(0, len(self.dataset_indices), self.batch_size):
                end_idx = min(start_idx + self.batch_size, len(self.dataset_indices))
                batch_indices = self.dataset_indices[start_idx:end_idx]
                batch_counter += 1
                yield self.get_data_from_indices(batch_indices)
        else:
            all_indices = self.dataset_indices  
            all_probabilities = self.sample_probabilities   

            total_sum = all_probabilities.sum()
            all_probabilities /= total_sum

            total_samples = len(self.dataset)
            num_batches = np.ceil(total_samples / self.batch_size).astype(int)

            for _ in range(num_batches):
                num_required = self.batch_size 
                if num_required > 0:
                    chosen_indices = np.random.choice(all_indices, size=num_required, replace=False, p=all_probabilities)
                else:
                    chosen_indices = []
                batch_indices = list(chosen_indices)
                yield self.get_data_from_indices(batch_indices)
                
    def get_data_from_indices(self, indices):
        x, y = zip(*[self.dataset[i] for i in indices])
        return indices, torch.stack(x), torch.stack(y)
    def set_epoch(self, epoch):
        self.current_epoch = epoch

# Uncertain

In [ ]:
def update_H(H, y_pred, ids, max_history_length=5):
    y_pred_numpy = y_pred.detach().cpu().numpy()
    for i, idx in enumerate(ids):
        if idx not in H:
 # 为每个样本创建一个新的deque，限制最大长度
            H[idx] = deque(maxlen=max_history_length) 
# 添加新的预测，如果deque已满，最早的预测将自动被移除
        H[idx].append(y_pred_numpy[i])  
    return H
def update_E(H,E,ids,label_dim):
    
    for idx in ids:
        current_predictions_history = np.array(H[idx])
        last_row_index = len(current_predictions_history) - 1
        for j in range(label_dim):
# 计算差分（除了第一个元素，因为差分是相邻元素的差）  
            diffs = np.abs(np.diff(current_predictions_history[:, j]))
            mean_diffs = np.sum(diffs)/len(diffs)
            current_entropy = -1 / np.log(2) * (current_predictions_history[last_row_index][j]  * np.log(current_predictions_history[last_row_index][j]) + (1 - current_predictions_history[last_row_index][j] ) * np.log(1 -current_predictions_history[last_row_index][j] ))
            E[idx,j]=1/2*mean_diffs+1/2* current_entropy
    return E

def update_U(E, U, epoch):
    E[E > 1] = 1
    if epoch>=5:
        bins = np.array([0, 0.2, 0.4, 0.6, 0.8, 1.0])
        discrete_values = np.array([0.1, 0.3, 0.5, 0.7, 0.9])
        indices = np.digitize(E, bins) - 1
        indices = np.clip(indices, 0, len(discrete_values) - 1)
        E = discrete_values[indices]
#         I=np.abs(calculate_pearson_matrix(E))
        
    else:
        I = np.ones((E.shape[1],E.shape[1]))
#     if epoch==30:
#         np.save('yahoo-Business1I(30).npy', I)
    I = np.ones((E.shape[1],E.shape[1]))
    U = np.dot(E, I)
    w = np.sum(U, axis=1)
    return w



def calculate_probabilities(U, epoch, e_end, e0, se0):
    # Delta is set to 1/N
    N=len(U)
    Delta = 1 / N 
    # Exponentially decay the selection pressure as training progresses
    se = se0 * (np.exp(np.log(1 / se0) / (e_end - e0)))**(epoch - e0)
    U_min = np.min(U)
    U_max = np.max(U)
    U_normalized = (U - U_min) / (U_max - U_min)
    quantized_indices = np.ceil((1 - U_normalized) / Delta).astype(int)
#     pdb.set_trace()
    # Calculate the exponent for the sampling probability
    exponent = -np.log(se) / N * quantized_indices 
    # Calculate the unnormalized probabilities
    unnormalized_probabilities = np.exp(exponent)
    return unnormalized_probabilities


def calculate_mutual_information_matrix(data):
    n, q = data.shape
    mi_matrix = np.zeros((q, q))
    for i in range(q):
        for j in range(q):
            if i == j:
                mi_matrix[i, j] = 1  # 对角线置为1
            else:
                mi_matrix[i, j] = mutual_info_classif(data[:, i].reshape(-1, 1), data[:, j])[0]
                mi_matrix[i, j] = max(mi, 0)
    return mi_matrix

def calculate_pearson_matrix(data):
    pearson_matrix = np.corrcoef(data, rowvar=False)
    return pearson_matrix

def entropy_based_mutual_information_matrix(data):
    n, q = data.shape
    mi_matrix = np.zeros((q, q))
    
    for i in range(q):
        for j in range(i, q):
            if i == j:
                mi_matrix[i, j] = 1
            else:
                mi = entropy_based_mutual_information(data[:, i], data[:, j])
                mi_matrix[i, j] = mi
                mi_matrix[j, i] = mi
    
    # Normalize each row by its maximum value
    row_max = mi_matrix.max(axis=1, keepdims=True)
    mi_matrix = mi_matrix / row_max
    
    return mi_matrix

def entropy_based_mutual_information(x, y):
    # Calculate mutual information between x and y using entropy-based methods
    # Ensure x and y are flattened to 1D arrays
    x = np.ravel(x)
    y = np.ravel(y)
    
    # Calculate individual entropies
    h_x = entropy(np.histogram(x, bins='auto')[0])
    h_y = entropy(np.histogram(y, bins='auto')[0])
    
    # Calculate joint entropy
    joint_hist, _, _ = np.histogram2d(x, y, bins='auto')
    joint_entropy = entropy(joint_hist.ravel())
    
    # Mutual information
    mi = h_x + h_y - joint_entropy
    return mi

class CustomDataLoader:
    def __init__(self, dataset, net, configs,H,E,U,sample_probabilities,warm_epoch):
        self.shuffle = False
        self.dataset = dataset
        self.net = net
        self.batch_size = configs['batch_size']
        self.sample_probabilities =sample_probabilities
        self.dataset_indices=list(range(len(self.dataset)))
        self.warm_epoch=warm_epoch   
        
        self.H=H
        self.E=E
        self.U=U    
                              
    def __iter__(self):
        if self.current_epoch < self.warm_epoch:
            batch_counter = 0
            for start_idx in range(0, len(self.dataset_indices), self.batch_size):
                end_idx = min(start_idx + self.batch_size, len(self.dataset_indices))
                batch_indices = self.dataset_indices[start_idx:end_idx]
                batch_counter += 1
                yield self.get_data_from_indices(batch_indices)
        else:
            all_indices = self.dataset_indices  
            all_probabilities = self.sample_probabilities   

            total_sum = all_probabilities.sum()
            all_probabilities /= total_sum

            total_samples = len(self.dataset)
            num_batches = np.ceil(total_samples / self.batch_size).astype(int)

            for _ in range(num_batches):
                num_required = self.batch_size 
                if num_required > 0:
                    chosen_indices = np.random.choice(all_indices, size=num_required, replace=False, p=all_probabilities)
                else:
                    chosen_indices = []
                batch_indices = list(chosen_indices)
                yield self.get_data_from_indices(batch_indices)
                
    def get_data_from_indices(self, indices):
        x, y = zip(*[self.dataset[i] for i in indices])
        return indices, torch.stack(x), torch.stack(y)
    def set_epoch(self, epoch):
        self.current_epoch = epoch

In [9]:
def training(configs,test_epoch):
    net=CLIFModel(configs).to(device)
    num_epochs =configs['epoch']
    batch_size = configs['batch_size']
    card=int(configs['card'])
#     card=configs['num_classes']
    lr = 1e-4
    
    ins_dim=configs['num_ins']
    label_dim=configs['num_classes']
    
    weight_decay = 1e-4  
    betas = (0.9, 0.999) 

    optimizer = torch.optim.Adam(net.parameters(), lr=lr,  
                           betas=betas, weight_decay=weight_decay)
    writer = SummaryWriter(comment=f'{dataname}')

    sample_probabilities=[]
    sample_indices=list(range(ins_dim))
    H = {idx: deque(maxlen=5) for idx in sample_indices}
    E= np.zeros((ins_dim, label_dim))
    U = np.zeros(ins_dim)
    U_all= np.zeros(ins_dim)
    warmup_epochs =5
    custom_dataloader = CustomDataLoader(train_dataset, net=net, configs=configs,H=H,E=E,U=U,sample_probabilities=sample_probabilities,warm_epoch=5)
    validation_dataset, test_dataset_new = train_test_split(test_dataset, test_size=0.5, random_state=42)   
    test_dataloader = DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False, num_workers=0)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    best_auc = 0
    best_model_state = None
    epoch_losses_train = []
    all_loss_list=[]
    
    
    total_steps = num_epochs * int(configs['num_ins']/batch_size)
    warmup_steps = warmup_epochs * int(configs['num_ins']/batch_size)
    global_step = 0
    def update_learning_rate(optimizer, global_step, warmup_steps=warmup_steps, base_lr=lr):
        if global_step < warmup_steps:
            lr = base_lr * (global_step / warmup_steps)
        else:
            lr = base_lr
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr 
    all_loss_list=[]   
    mean_U=[]
    for epoch in range(num_epochs+1): 
        net.train()
        batch_counter = 0   
        loss_tracker = 0.0
        loss_tracker2=0.0
        
        custom_dataloader.set_epoch(epoch) 
        for idx, x, y in custom_dataloader:
            optimizer.zero_grad()
            outputs = net(x)
            loss_dict = net.loss_function_train(outputs, y)
            loss = loss_dict['Loss']
#             分析所有样本的loss分布          
            if epoch == test_epoch:
                individual_losses1 = net._loss_per_label(outputs[configs['out_index']], y)
                for i in range(individual_losses1.shape[0]):
                    all_loss_list.append(round(individual_losses1[i].item(), 3))
            loss.backward()
            optimizer.step()
            global_step += 1
            update_learning_rate(optimizer, global_step) 
            custom_dataloader.H=update_H(custom_dataloader.H, torch.sigmoid(outputs[0]), idx)
            custom_dataloader.E=update_E(custom_dataloader.H, custom_dataloader.E,idx,label_dim)
            batch_counter+=1 
        custom_dataloader.U=update_U(custom_dataloader.E, custom_dataloader.U, epoch)
        custom_dataloader.sample_probabilities=calculate_probabilities(custom_dataloader.U, epoch, 100, 0, 10) 
        mean_U.append(np.sum(custom_dataloader.U)/len(custom_dataloader.U))
        if epoch==99:
            np.save('bibtex_mean_U.npy', mean_U)
#             np.save('yahoo-Business1E(30)_1.npy', custom_dataloader.E)
#             np.save('yahoo-Business1U(30)_1.npy', custom_dataloader.U)  
        writer.add_scalar('train/loss', loss_tracker, epoch)
        epoch_losses_train.append(loss_tracker /batch_counter)
        net.eval()
        y_true_list = []
        y_scores_list = []
        loss_tracker = 0.0
        batch_counter = 0
        with torch.no_grad():
            for x, y in validation_dataloader:
                _, y_pred=net.predict(x)
                y_true_list.append(y.cpu().numpy())
                y_scores_list.append(y_pred.cpu().numpy())
                
                outputs = net(x)
                loss_dict = net.loss_function_train(outputs, y)
                loss = loss_dict['Loss']
                loss_tracker += loss.item()
                batch_counter += 1
        y_true = np.vstack(y_true_list)
        y_scores = np.vstack(y_scores_list)

        auc = macro_averaging_auc(y_true,y_scores, y_scores)
#         writer.add_scalar('val/auc', auc, epoch)

        if auc > best_auc:
            best_auc = auc
            best_epoch=epoch
            best_model_state = net.state_dict().copy()
    net.load_state_dict(best_model_state)
    mets = eval_metrics(net, [macro_f1, micro_f1, macro_averaging_auc, ranking_loss, hamming_loss, one_error], test_dataset_new, configs['out_index'],configs['batch_size'],torch.device('cuda'))
    return all_loss_list


# path_to_arff_files = ["emotions","scene","yeast", "Corel5k","rcv1subset1","rcv1subset2","rcv1subset3","yahoo-Business1","yahoo-Arts1","bibtex",'tmc2007','enron','cal500','LLOG-F']
# label_counts = [6, 6,14,374,101,101,101,28,25,159,22,53,174,75]
# select_feature=[1,1,1,1,0.02,0.02,0.02,0.05,0.05,1,0.01,1,1,1]

path_to_arff_files = ["bibtex"]
label_counts = [159]
select_feature=[1]
sample_loss_dict={}


for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[idx],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
        )
    X,feature_names=FeatureSelect(X,select_feature[idx])  
    y,label_names=LabelSelect(y)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    warm_epoch=10
    print(dataname)
    k_fold = IterativeStratification(n_splits=5,order=1,random_state=42)
    dicts=[]
    for idxx,(train,test) in enumerate(k_fold.split(X,y)):
        if idxx==4:
            train_dataset = TensorDataset(torch.tensor(X[train], device=device, dtype=torch.float),torch.tensor(y[train], device=device,dtype=torch.float))
            test_dataset = TensorDataset(torch.tensor(X[test], device=device, dtype=torch.float), torch.tensor(y[test], device=device, dtype=torch.float))    
            configs=CFG('CLIF',X[train],y[train]).getconfig()
            result=training(configs,30)
            sample_loss_dict[dataname]=result
#             epoch_loss_dict_train[dataname]=epoch_losses_train
#             epoch_loss_dict_val[dataname]=epoch_losses_val
#             epoch_auc_dict_val[dataname]=epoch_auc_val
#             print(best_epoch)
# with open('epoch_loss_dict_train0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_train, f)
# with open('epoch_loss_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_val, f)
# with open('epoch_auc_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_auc_dict_val, f)
# with open('batch_loss_dict_train1.pickle', 'wb') as f:
#     pickle.dump(batch_loss_dict_train, f)
# with open('Figure/sample_loss_dict_Uncertain(30).pickle', 'wb') as f:
#     pickle.dump(sample_loss_dict, f)

bibtex


In [ ]:
path_to_arff_files = ["bibtex"]
label_counts = [159]
select_feature=[1]
sample_loss_dict={}


for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[idx],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
        )
    X,feature_names=FeatureSelect(X,select_feature[idx])  
    y,label_names=LabelSelect(y)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    warm_epoch=10
    print(dataname)
    k_fold = IterativeStratification(n_splits=5,order=1,random_state=42)
    dicts=[]
    for idx,(train,test) in enumerate(k_fold.split(X,y)):
        if idx==4:
            train_dataset = TensorDataset(torch.tensor(X[train], device=device, dtype=torch.float),torch.tensor(y[train], device=device,dtype=torch.float))
            test_dataset = TensorDataset(torch.tensor(X[test], device=device, dtype=torch.float), torch.tensor(y[test], device=device, dtype=torch.float))    
            configs=CFG('CLIF',X[train],y[train]).getconfig()
            result=training(configs,30)
            sample_loss_dict[dataname]=result
#             epoch_loss_dict_train[dataname]=epoch_losses_train
#             epoch_loss_dict_val[dataname]=epoch_losses_val
#             epoch_auc_dict_val[dataname]=epoch_auc_val
#             print(best_epoch)
# with open('epoch_loss_dict_train0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_train, f)
# with open('epoch_loss_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_loss_dict_val, f)
# with open('epoch_auc_dict_val0.pickle', 'wb') as f:
#     pickle.dump(epoch_auc_dict_val, f)
# with open('batch_loss_dict_train1.pickle', 'wb') as f:
#     pickle.dump(batch_loss_dict_train, f)
# with open('Figure/sample_loss_dict_Uncertain(70).pickle', 'wb') as f:
#     pickle.dump(sample_loss_dict, f)